# Capstone Project - Car Accident Severity

## Table of contents
* [Introduction](#introduction)
* [Data Understanding and Preparation](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction <a name="introduction"></a>

When it comes to car accidents, there can be a variety of external factors that contribute and lead to the causes. As one can imagine, road intersections might be the hot spots where accidents happen. Also, accidents may happen more frequently during a raining day due to lack of vision. A combination of such external factors may lead to road accidents with different severity. It is of high interest to predict severity of road accidents based on the readily observable and attainable data of external conditions, so that first responders are aware of the type of accidents they are dealing with.

In this study, we will examine across a large amount of car accidents with the power of data science, and correlate the severity of the accidents with their attributes, such as locations, road conditions, weather, and etc. 

## Data Understanding and Preparation <a name="data"></a>

### Data Sources

In this study, we will use the dataset provided by the course of Applied Data Science Capstone project. This data set contains nearly 200000 cases of car accidents, which is sufficient for our study. 

### Data Selection and Cleaning

This dataset has a few problems and need cleaning.

#### Duplicate Columns

The target variable that we try to predict in this study is the severity code. This variable has two values, 1 and 2, whereas a value of 1 suggests property damage only, and a value of 2 indicates injury is involved. A glance at this dataset shows that there are two columns that contain data of this variable. Thus, we start by dropping one of the columns.

In [17]:
import pandas as pd
import numpy as np
import math

In [76]:
df = pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

The above list shows the column names of the data set. Notice that there are two variables 'SEVERITYCODE' and 'SEVERITYCODE.1'. These two variables are duplicates. Here we drop 'SEVERITYCODE.1'.

In [77]:
df.drop('SEVERITYCODE.1',axis=1,inplace=True)

In [35]:
df.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYDESC', 'COLLISIONTYPE', 'PERSONCOUNT',
       'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE', 'INCDTTM',
       'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC', 'INATTENTIONIND',
       'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT',
       'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY',
       'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

####  Feature Selection

This vast dataset contains information that is clearly not related to the cause of a road accident, such as OBJECTID, INCKEY, and etc. Here we only keep meaningful data that potentially have an impact on the cause of a road accidents, which include the following: 
* Location of the accident: 'X', 'Y'
* Is the location a intersection or a block: 'ADDRTYPE'
* Weather condition: 'WEATHER'
* Under influence of achohol: 'UNDERINFL'
* Road Condition: 'ROADCOND'
* Light Condition: 'LIGHTCOND'
* Speeding or not: 'SPEEDING'


In [78]:
df = df[['SEVERITYCODE', 'X', 'Y', 'ADDRTYPE', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'SPEEDING']]

In [37]:
df.head(15)

,SEVERITYCODE,X,Y,ADDRTYPE,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,-122.323148,47.703140,Intersection,N,Overcast,Wet,Daylight,NaN
1,1,-122.347294,47.647172,Block,0,Raining,Wet,Dark - Street Lights On,NaN
2,1,-122.334540,47.607871,Block,0,Overcast,Dry,Daylight,NaN
3,1,-122.334803,47.604803,Block,N,Clear,Dry,Daylight,NaN
4,2,-122.306426,47.545739,Intersection,0,Raining,Wet,Daylight,NaN
5,1,-122.387598,47.690575,Intersection,N,Clear,Dry,Daylight,NaN
6,1,-122.338485,47.618534,Intersection,0,Raining,Wet,Daylight,NaN
7,2,-122.320780,47.614076,Intersection,N,Clear,Dry,Daylight,NaN
8,1,-122.335930,47.611904,Block,0,Clear,Dry,Daylight,NaN
9,2,-122.384700,47.528475,Intersection,0,Clear,Dry,Daylight,NaN


#### Missing Values

Notice that the above dataset contains many values of NaN. These values need to be dealt with in different ways. 

* The location data 'X' and 'Y' have a few rows of missing data. In this case, the entire rows are dropped. The same approach is applied to the other features where the number of NaN values contributes to only a small portion of the whole columns. 
* An exception is the data contained in 'SPEEDING'. Due to the data format contained in the raw spreadsheet, only a speeding case has an entry 'Y', the rest are missing values. In this case we replace the NaN values with 0.

In [79]:
df['SPEEDING'].fillna(value=0,axis=0,inplace=True)
df.dropna(inplace=True)
df.head(15)

,SEVERITYCODE,X,Y,ADDRTYPE,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING
0,2,-122.323148,47.703140,Intersection,N,Overcast,Wet,Daylight,0
1,1,-122.347294,47.647172,Block,0,Raining,Wet,Dark - Street Lights On,0
2,1,-122.334540,47.607871,Block,0,Overcast,Dry,Daylight,0
3,1,-122.334803,47.604803,Block,N,Clear,Dry,Daylight,0
4,2,-122.306426,47.545739,Intersection,0,Raining,Wet,Daylight,0
5,1,-122.387598,47.690575,Intersection,N,Clear,Dry,Daylight,0
6,1,-122.338485,47.618534,Intersection,0,Raining,Wet,Daylight,0
7,2,-122.320780,47.614076,Intersection,N,Clear,Dry,Daylight,0
8,1,-122.335930,47.611904,Block,0,Clear,Dry,Daylight,0
9,2,-122.384700,47.528475,Intersection,0,Clear,Dry,Daylight,0


#### Data Formatting and Filtering

In [68]:
df.dtypes

ADDRTYPE         object
Dry             float64
Ice             float64
LIGHTCOND        object
ROADCOND         object
SEVERITYCODE    float64
SPEEDING         object
UNDERINFL        object
WEATHER          object
X               float64
Y               float64
dtype: object

Many features in the above dataset has string values. To prepare this dataset for the downstream analysis, we need to replace the categorical values with quantitative values. Here we use one-shot binning to convert categorical data into a number of indicator columns. For example, the 'ROADCOND' column is converted into 9 separate columns - 'Dry', 'Ice', and etc, each indicating whether an accident happened under such road conditions, as shown below. 

In [80]:
df_wea=pd.get_dummies(df['ROADCOND'])
df_wea.head()

,Dry,Ice,Oil,Other,Sand/Mud/Dirt,Snow/Slush,Standing Water,Unknown,Wet
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1


A further investigation shows that 'Dry' and 'Wet' conditions contribute to 91% of the total number of cases. To simplify the dataset, we only keep 'Dry' and 'Wet' columns, and drop the rest.

In [81]:
df['ROADCOND'].value_counts()

Dry               121871
Wet                46009
Unknown            13795
Ice                 1174
Snow/Slush           984
Other                116
Standing Water       102
Sand/Mud/Dirt         63
Oil                   53
Name: ROADCOND, dtype: int64

In [82]:
df_wea=df_wea[['Dry','Ice']]
df_wea.head()

,Dry,Ice
0,0,0
1,0,0
2,1,0
3,1,0
4,0,0


The two columns are concatenated with the dataset, and the original column 'ROADCOND' is dropped.

In [87]:
df_wea.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df,df_wea],axis=1)


In [91]:
df.drop(['ROADCOND'],axis=1,inplace=True)

In [92]:
df.head()

,SEVERITYCODE,X,Y,ADDRTYPE,UNDERINFL,WEATHER,LIGHTCOND,SPEEDING,Dry,Ice
0,2,-122.323148,47.703140,Intersection,N,Overcast,Daylight,0,0,0
1,1,-122.347294,47.647172,Block,0,Raining,Dark - Street Lights On,0,0,0
2,1,-122.334540,47.607871,Block,0,Overcast,Daylight,0,1,0
3,1,-122.334803,47.604803,Block,N,Clear,Daylight,0,1,0
4,2,-122.306426,47.545739,Intersection,0,Raining,Daylight,0,0,0


The above approach is also applied to columns 'ADDRTYPE', 'WEATHER', and 'LIGHTCOND'. 

In [93]:
df_add=pd.get_dummies(df['ADDRTYPE'])
df_add.head()

,Block,Intersection
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1


In [94]:
df_add.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df,df_add],axis=1)

In [96]:
df.drop(['ADDRTYPE'],axis=1,inplace=True)
df.head()

,SEVERITYCODE,X,Y,UNDERINFL,WEATHER,LIGHTCOND,SPEEDING,Dry,Ice,Block,Intersection
0,2,-122.323148,47.703140,N,Overcast,Daylight,0,0,0,0,1
1,1,-122.347294,47.647172,0,Raining,Dark - Street Lights On,0,0,0,1,0
2,1,-122.334540,47.607871,0,Overcast,Daylight,0,1,0,1,0
3,1,-122.334803,47.604803,N,Clear,Daylight,0,1,0,1,0
4,2,-122.306426,47.545739,0,Raining,Daylight,0,0,0,0,1


In [98]:
df_wea=pd.get_dummies(df['WEATHER'])
df['WEATHER'].value_counts()

Clear                       108833
Raining                      31987
Overcast                     27105
Unknown                      13846
Snowing                        888
Other                          765
Fog/Smog/Smoke                 553
Sleet/Hail/Freezing Rain       112
Blowing Sand/Dirt               49
Severe Crosswind                24
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [99]:
df_wea=df_wea[['Clear','Raining','Overcast']]

In [100]:
df_wea.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df,df_wea],axis=1)

In [101]:
df.drop(['WEATHER'],axis=1,inplace=True)
df.head()

,SEVERITYCODE,X,Y,UNDERINFL,LIGHTCOND,SPEEDING,Dry,Ice,Block,Intersection,Clear,Raining,Overcast
0,2,-122.323148,47.703140,N,Daylight,0,0,0,0,1,0,0,1
1,1,-122.347294,47.647172,0,Dark - Street Lights On,0,0,0,1,0,0,1,0
2,1,-122.334540,47.607871,0,Daylight,0,1,0,1,0,0,0,1
3,1,-122.334803,47.604803,N,Daylight,0,1,0,1,0,1,0,0
4,2,-122.306426,47.545739,0,Daylight,0,0,0,0,1,0,1,0


In [102]:
df_wea=pd.get_dummies(df['LIGHTCOND'])
df['LIGHTCOND'].value_counts()

Daylight                    113522
Dark - Street Lights On      47250
Unknown                      12416
Dusk                          5763
Dawn                          2422
Dark - No Street Lights       1450
Dark - Street Lights Off      1145
Other                          188
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [103]:
df_wea=df_wea[['Daylight','Dark - Street Lights On']]
df_wea.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df = pd.concat([df,df_wea],axis=1)
df.drop(['LIGHTCOND'],axis=1,inplace=True)
df.head()

,SEVERITYCODE,X,Y,UNDERINFL,SPEEDING,Dry,Ice,Block,Intersection,Clear,Raining,Overcast,Daylight,Dark - Street Lights On
0,2,-122.323148,47.703140,N,0,0,0,0,1,0,0,1,1,0
1,1,-122.347294,47.647172,0,0,0,0,1,0,0,1,0,0,1
2,1,-122.334540,47.607871,0,0,1,0,1,0,0,0,1,1,0
3,1,-122.334803,47.604803,N,0,1,0,1,0,1,0,0,1,0
4,2,-122.306426,47.545739,0,0,0,0,0,1,0,1,0,1,0


The column 'UNDERINFL' contains mixed values of Y, 0, and N. Both N and 0 indicate the driver is not influenced by achohol. Here we convert values of N to integer 0, and Y to integer 1. We apply the same approach to column 'SPEEDING'

In [106]:
df['UNDERINFL'].replace(to_replace='N',value=0,inplace=True)
df['UNDERINFL'].replace(to_replace='Y',value=1,inplace=True)


In [110]:
df['SPEEDING'].replace(to_replace='Y',value=1,inplace=True)


And finally, upon checking the data types of each column, we notice that column 'UNDERINFL' have a type of 'object'. We need to cast these two variables into integers for further analysis.

In [114]:
df['UNDERINFL'].replace(to_replace='1',value=1,inplace=True)

In [115]:
df.dtypes

SEVERITYCODE                 int64
X                          float64
Y                          float64
UNDERINFL                    int64
SPEEDING                     int64
Dry                          uint8
Ice                          uint8
Block                        uint8
Intersection                 uint8
Clear                        uint8
Raining                      uint8
Overcast                     uint8
Daylight                     uint8
Dark - Street Lights On      uint8
dtype: object

In [116]:
df.head()

,SEVERITYCODE,X,Y,UNDERINFL,SPEEDING,Dry,Ice,Block,Intersection,Clear,Raining,Overcast,Daylight,Dark - Street Lights On
0,2,-122.323148,47.703140,0,0,0,0,0,1,0,0,1,1,0
1,1,-122.347294,47.647172,0,0,0,0,1,0,0,1,0,0,1
2,1,-122.334540,47.607871,0,0,1,0,1,0,0,0,1,1,0
3,1,-122.334803,47.604803,0,0,1,0,1,0,1,0,0,1,0
4,2,-122.306426,47.545739,0,0,0,0,0,1,0,1,0,1,0


The above dataset is not cleaned, formated, and filtered for modeling and downstream analysis.